### We can construct a mosaic of nearby tiles using this method: https://www.kaggle.com/c/planet-understanding-the-amazon-from-space/discussion/36738

# Construct image tile mosaic
---

In [ ]:
# first pass, construct a list of image strips
import matplotlib
from matplotlib import pyplot as plt
import os
import numpy as np

train_path = 'data/train-jpg/'

num_images = len(os.listdir(train_path))

image_strip_list = []

for file in os.listdir(train_path):
    # load image as numpy array
    image = matplotlib.image.imread(train_path + file)

    # drop alpha channel
    image = image[:, :, :3]

    # 1) Get vectors of all edges of images. So 4 vectors of size 3*256 for each image.
    left = image[:, 0, :]
    right = image[:, 255, :]
    top = image[0, :, :]
    bottom = image[255, :, :]

    # append to list containing 4-tuples of image strips: (left, right, top, bottom)
    image_strip_list.append( (left, right, top, bottom) )

In [ ]:
# second pass, find neighboring images based on their strips

# In this algorithm, we assume that images have not been rotated

# 2) For each image find difference between top vector and bottom vector of all other images. Take the minimum. It will be the TOP neighbour. Do the same for BOTTOM, LEFT and RIGHT.

# my idea: construct a dictionary where: 
#   * key is an integer representing the image name
#   * value is a 4-list of 2-lists
#       * 4-list: [ [left, diff.], [right, diff.], [top, diff.], [bottom, diff.] ]
# The 4-list represents the current best neighboring images and their
# difference (after subtracting two image strips)

INIT_DIFF = 999999999

image_mosaic_dict = {i:[[None, INIT_DIFF], [None, INIT_DIFF], [None, INIT_DIFF], [None, INIT_DIFF]] for i in range(num_images)}

for image_index, image in enumerate(image_strip_list):

    # iterate over image strip sides
    for side_index, side in enumerate(image):

        # check to see if side already has a neighbor
        neighbor = image_mosaic_dict[image_index][side_index][0]
        difference = image_mosaic_dict[image_index][side_index][1]
        if (neighbor, difference) != (None, INIT_DIFF):
            continue
        else:

            # side is left, search for rights
            if side_index == 0:
                for possible_index, (_, possible_right, _, _) in enumerate(image_strip_list):

                    # if difference is greater, take the new side
                    new_difference = np.abs(np.sum(side) - np.sum(possible_right))
                    if new_difference < difference:
                        # update current side info
                        image_mosaic_dict[image_index][side_index][0] = possible_index
                        image_mosaic_dict[image_index][side_index][1] = new_difference

                        # update new neighboring side info
                        image_mosaic_dict[possible_index][1][0] = image_index
                        image_mosaic_dict[possible_index][1][1] = new_difference

            # side is right, search for lefts
            if side_index == 1:
                for possible_index, (possible_left, _, _, _) in enumerate(image_strip_list):

                    # if difference is greater, take the new side
                    new_difference = np.abs(np.sum(side) - np.sum(possible_left))
                    if new_difference < difference:
                        # update current side info
                        image_mosaic_dict[image_index][side_index][0] = possible_index
                        image_mosaic_dict[image_index][side_index][1] = new_difference

                        # update new neighboring side info
                        image_mosaic_dict[possible_index][0][0] = image_index
                        image_mosaic_dict[possible_index][0][1] = new_difference

            # side is top, search for bottoms
            if side_index == 2:
                for possible_index, (_, _, _, possible_bottom) in enumerate(image_strip_list):

                    # if difference is greater, take the new side
                    new_difference = np.abs(np.sum(side) - np.sum(possible_bottom))
                    if new_difference < difference:
                        # update current side info
                        image_mosaic_dict[image_index][side_index][0] = possible_index
                        image_mosaic_dict[image_index][side_index][1] = new_difference

                        # update new neighboring side info
                        image_mosaic_dict[possible_index][3][0] = image_index
                        image_mosaic_dict[possible_index][3][1] = new_difference

            # side is bottom, search for tops
            if side_index == 3:
                for possible_index, (_, _, possible_top, _) in enumerate(image_strip_list):

                    # if difference is greater, take the new side
                    new_difference = np.abs(np.sum(side) - np.sum(possible_top))
                    if new_difference < difference:
                        # update current side info
                        image_mosaic_dict[image_index][side_index][0] = possible_index
                        image_mosaic_dict[image_index][side_index][1] = new_difference

                        # update new neighboring side info
                        image_mosaic_dict[possible_index][2][0] = image_index
                        image_mosaic_dict[possible_index][2][1] = new_difference

In [ ]:
                        image_mosaic_dict[possible_index][2][0] = image_index
                        image_mosaic_dict[possible_index][2][1] = new_difference

In [ ]:
# PEOPLE ON THE KAGGLE FORUMS SAY THAT, (in general), THE .tiff IMAGES WERE NOT 
# USEFUL. That might be the same case here.

# 3) Using the same algorithm get neighbours for all TIFF images.

# 4) Remove neighbours which is different for JPG and TIFF images.

# 5) Use diagonals to additional improvements. For example check that diagonal element have same neighbours as central one without conflicts and so on…

# 6) Use CNN averaged predictions for 4 or 8 neighbours as features for central element

# Preprocess data
---

In [32]:
# extract labels from training data
import pandas as pd

train_data = pd.read_csv('data/train_v2.csv/train_v2.csv')
print(train_data['tags'].values)

unique_labels = []
for line in train_data['tags'].values:
    for label in line.split():
        if label not in unique_labels:
            unique_labels.append(label)
            print(label)

print(len(unique_labels))

['haze primary' 'agriculture clear primary water' 'clear primary' ...
 'agriculture clear primary' 'agriculture clear primary road'
 'agriculture cultivation partly_cloudy primary']
haze
primary
agriculture
clear
water
habitation
road
cultivation
slash_burn
cloudy
partly_cloudy
conventional_mine
bare_ground
artisinal_mine
blooming
selective_logging
blow_down
17


# Construct model
---

# Train model
---

# View results
---